In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import pandas as pd
from pandas import Timestamp
from datetime import datetime, timedelta
import pandas as pd
from datetime import datetime, timedelta
import pandas as pd
import pyodbc
import pandas as pd# select command


In [2]:
server = 'fidosqldb.risk.regn.net,50750'
database = 'red' # enter database name
username = 'pasl_fidoanalytics'
password = 'H8o0T1saPuC' # add appropriate driver name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


query = ''' 
select o.date_sk, c.ins_control_nbr, c.ins_control_name, c.ins_company_name, c.ins_company_nbr,  p.product_name, s.search_type_name,s.search_type_id, o.search_type_sk, g.geography_sk, g.country, g.state_province_name, 
sum(o.unit_count) as total_units

FROM  fact_mvr_ops o
left join dim_search_type s on o.search_type_sk=s.search_type_sk
left join dim_date d on d.date_sk=o.date_sk
left join dim_geography g on o.geography_sk=g.geography_sk
left join dim_customer_account c on c.customer_account_sk = o.customer_account_sk
left join dim_product p on p.product_sk = o.product_sk
where 
c.ins_company_name not in ('', 'Lexisnexis Test Accts')  and o.product_sk = 84 
and c.mbs_product_id = 9 
and o.scenario_sk in (1,2)
and s.search_type_name not in ('MVR FLORIDA 1-MONTH CUSTOM', 
                                 'MVR GEORGIA 1-MONTH',
                                 'MVR IDAHO 1-MONTH',
                                'MVR IOWA 1-MONTH CUSTOM',
                                'MVR KANSAS 1-MONTH',
                                 'MVR MAINE 1-MONTH CUSTOM',
                                  'MVR MICHIGAN 1-MONTH',
                                   'MVR MONTANA 1-MONTH',
                                  'MVR NEBRASKA 1-MONTH',
                                  'MVR NEVADA 1-MONTH',
                                   'MVR NEW HAMPSHIRE 1-MONTH CUSTOM',
                                    'MVR NEW YORK 1-MONTH',
                                   'MVR OHIO 1-MONTH CUSTOM',
                                  'MVR UTAH 1-MONTH',
                                  'MVR VERMONT 1-MONTH CUSTOM')
and d.date_sk >= '20200427' and d.date_sk <= '20200726'  
group by o.date_sk, c.ins_control_nbr, c.ins_control_name ,   c.ins_company_name, c.ins_company_nbr,  p.product_name, s.search_type_name,s.search_type_id, o.search_type_sk,g.geography_sk, g.country, g.state_province_name

''';


data = pd.read_sql(query, cnxn)
data.head()

,date_sk,ins_control_nbr,ins_control_name,ins_company_name,ins_company_nbr,product_name,search_type_name,search_type_id,search_type_sk,geography_sk,country,state_province_name,total_units
0,20200514,03079,Liberty Mut Ins Co,Liberty Mut Ins Co,03079,Motor Vehicle Records,MVR NEW JERSEY 36-MONTH CUSTOM ONLINE,4123,29519,32,USA,New Jersey,346.0
1,20200523,03027,Farmers Ins Grp,Bristol West Ins Grp,03785,Motor Vehicle Records,MVR DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM...,6435,58389,37,USA,Oklahoma,2.0
2,20200529,92955,Massachusetts Mutual Life Insurance Company,Massachusetts Mutual Life Insurance Company,92955,Motor Vehicle Records,MVR GEORGIA 36-MONTH CLEAR ONLINE,0302,28404,11,USA,Georgia,11.0
3,20200427,03113,Western National Mutual,American Freedom Ins,30426,Motor Vehicle Records,MVR ONLINE ACCESS,0004,28263,43,USA,Tennessee,1.0
4,20200429,03037,Hartford Ins Grp,Hartford Ins Grp,03037,Motor Vehicle Records,MVR DRIVER HISTORY DATABASE ONLINE AGENT,0568,28519,4,USA,Arizona,9.0


In [3]:
data.shape

(1151863, 13)

In [4]:


df = pd.DataFrame(data['date_sk'].tolist(), columns = ['date_int']) 
# Pandas has a it's own datetime libraries
# df['dttm'] = df.apply(pd.to_datetime(df['date_int'].astype(str)).values, axis=1)
df['date_string'] = df['date_int'].astype(str) #df.apply(lambda row: str(row.date_int), axis=1)#'Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [5]:
min(data['date_sk'])

20200427

In [6]:
max(data['date_sk'])

20200726

In [7]:
data.shape

(1151863, 13)

In [8]:
# add col to convert to datetime
# func = lambda row: datetime.strptime(df['date_string'], '%m%d%y'), axis=1
df['dttm'] = df['date_string'].apply(lambda x: datetime.strptime(x, '%Y%m%d'))

In [9]:
#https://docs.python.org/3/library/datetime.html#datetime.date.isocalendar
#The ISO year consists of 52 or 53 full weeks, and where a week starts on a Monday and ends on a Sunday.
df['week'] = df['dttm'].apply(lambda x: x.isocalendar()[1])

In [10]:
df.head()

,date_int,date_string,dttm,week
0,20200514,20200514,2020-05-14,20
1,20200523,20200523,2020-05-23,21
2,20200529,20200529,2020-05-29,22
3,20200427,20200427,2020-04-27,18
4,20200429,20200429,2020-04-29,18


In [11]:
max_week = max(df['week'])
max_week

30

In [12]:
### Below code is simple but takes long time to run, 

#df['week_format'] = df.apply(lambda row: "Prior-Wk-{}".format(max(df['week'])-row.week) if row.week != max(df['week']) else "curr_week", axis=1)

In [13]:
### The fast alternative for above one line of code
week_reverse = df['week']
week_reverse2 = max(df['week']) - df['week']
date_reverse_str = week_reverse2.astype(str)
prior = ['Prior-Wk-']*df.shape[0]          
df['week_format'] = prior + date_reverse_str
df['week_format'] [df['week_format'] =='Prior-Wk-0'] = 'Current-Wk'

<ipython-input-13-a4cf41365019>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['week_format'] [df['week_format'] =='Prior-Wk-0'] = 'Current-Wk'


In [14]:
df.head()

,date_int,date_string,dttm,week,week_format
0,20200514,20200514,2020-05-14,20,Prior-Wk-10
1,20200523,20200523,2020-05-23,21,Prior-Wk-9
2,20200529,20200529,2020-05-29,22,Prior-Wk-8
3,20200427,20200427,2020-04-27,18,Prior-Wk-12
4,20200429,20200429,2020-04-29,18,Prior-Wk-12


In [15]:
df['week_format'].value_counts()

Prior-Wk-1     91163
Current-Wk     90975
Prior-Wk-4     90856
Prior-Wk-6     90359
Prior-Wk-7     90327
Prior-Wk-2     90102
Prior-Wk-5     90085
Prior-Wk-10    89610
Prior-Wk-11    89329
Prior-Wk-12    89228
Prior-Wk-9     88915
Prior-Wk-3     80616
Prior-Wk-8     80298
Name: week_format, dtype: int64

In [16]:
data['week_format'] = df['week_format']
data.head()

,date_sk,ins_control_nbr,ins_control_name,ins_company_name,ins_company_nbr,product_name,search_type_name,search_type_id,search_type_sk,geography_sk,country,state_province_name,total_units,week_format
0,20200514,03079,Liberty Mut Ins Co,Liberty Mut Ins Co,03079,Motor Vehicle Records,MVR NEW JERSEY 36-MONTH CUSTOM ONLINE,4123,29519,32,USA,New Jersey,346.0,Prior-Wk-10
1,20200523,03027,Farmers Ins Grp,Bristol West Ins Grp,03785,Motor Vehicle Records,MVR DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM...,6435,58389,37,USA,Oklahoma,2.0,Prior-Wk-9
2,20200529,92955,Massachusetts Mutual Life Insurance Company,Massachusetts Mutual Life Insurance Company,92955,Motor Vehicle Records,MVR GEORGIA 36-MONTH CLEAR ONLINE,0302,28404,11,USA,Georgia,11.0,Prior-Wk-8
3,20200427,03113,Western National Mutual,American Freedom Ins,30426,Motor Vehicle Records,MVR ONLINE ACCESS,0004,28263,43,USA,Tennessee,1.0,Prior-Wk-12
4,20200429,03037,Hartford Ins Grp,Hartford Ins Grp,03037,Motor Vehicle Records,MVR DRIVER HISTORY DATABASE ONLINE AGENT,0568,28519,4,USA,Arizona,9.0,Prior-Wk-12


In [17]:
current_wk= pd.DataFrame(data[data['week_format'] == 'Current-Wk'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 
                                'ins_company_nbr','search_type_name', 'search_type_id','search_type_sk','state_province_name','geography_sk' ],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
current_wk.columns= ['Current_Wk_units']
current_wk

Current_Wk_units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT             2048           29007          California          5                          2.0
                                                                                                                                                                       Florida             10                         2.0
                                                                                                                                                                       Hawaii              12                         2.0
                                                                                                                                                                       Kansas              17                         1.0
                                                                                                                                                                       Missouri            25                         4.0
...                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR DRIVER HISTORY DATABASE ONLINE 2309           29072          New York            35                         4.0
                                                                                                      MVR EDIT REJECT ONLINE             0214           28362          Pennsylvania        39                         2.0
                                                                                                      MVR GEORGIA 36-MONTH ONLINE        0301           28403          Georgia             11                         1.0
                                                                                                      MVR ONLINE ACCESS                  0004           28263          New York            35                        83.0
                                                                                                                                                                       Vermont             47                         1.0

[28977 rows x 1 columns]

In [18]:
Prior_WK_1= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-1'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_1.columns= ['Prior_WK_1_Units']
Prior_WK_1

Prior_WK_1_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          Arkansas            3                          5.0
                                                                                                                                                                                       Hawaii              12                         2.0
                                                                                                                                                                                       Kansas              17                         2.0
                                                                                                                                                                                       Missouri            25                         2.0
                                                                                                                                                                                       New York            35                         4.0
...                                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Nevada              34                         1.0
                                                                                                                                                                                       Texas               44                         1.0
                                                                                                                                                                                       Virginia            46                         1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                         5.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                        62.0

[29050 rows x 1 columns]

In [19]:
Prior_WK_2= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-2'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_2.columns= ['Prior_WK_2_Units']
Prior_WK_2

Prior_WK_2_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          Arizona             4                         43.0
                                                                                                                                                                                       Florida             10                         1.0
                                                                                                                                                                                       Missouri            25                         4.0
                                                                                                                                                                                       New York            35                         2.0
                                                                                                                                                                                       North Carolina      28                         4.0
...                                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Texas               44                         1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                         4.0
                                                                                                                                                                                       Virginia            46                         1.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                        72.0
                                                                                                                                                                                       South Carolina      41                         1.0

[28886 rows x 1 columns]

In [20]:
frame0 = [current_wk,Prior_WK_1,Prior_WK_2 ]
result0 = pd.concat(frame0,axis=1, sort=False)
result0.head(10)

Current_Wk_units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          NaN   
                                                                                                                       Arkansas            3                          NaN   
                                                                                                                       California          5                          2.0   
                                                                                                                       Florida             10                         2.0   
                                                                                                                       Hawaii              12                         2.0   
                                                                                                                       Kansas              17                         1.0   
                                                                                                                       Missouri            25                         4.0   
                                                                                                                       New York            35                         3.0   
                                                                                                                       North Carolina      28                        25.0   
                                                                                                                       North Dakota        29                         NaN   

                                                                                                                                                         Prior_WK_1_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          NaN   
                                                                                                                       Arkansas            3                          5.0   
                                                                                                                       California          5                          NaN   
                                                                                                                       Florida             10                         NaN   
                                                                                                                       Hawaii              12                         2.0   
                                                                                                                       Kansas              17                         2.0   
                                                                                                                       Missouri            25                         2.0   
                                                                                                                       New York            35                         4.0   
                                                                                                                       North Carolina      28                         8.0   
                                                                                                                       North Dakota        29                         NaN   


In [21]:
result0.shape

(40415, 3)

In [22]:
Prior_WK_3= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-3'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_3.columns= ['Prior_WK_3_Units']
Prior_WK_3

Prior_WK_3_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          Arizona             4                          2.0
                                                                                                                                                                                       California          5                          1.0
                                                                                                                                                                                       Hawaii              12                         2.0
                                                                                                                                                                                       Missouri            25                         6.0
                                                                                                                                                                                       New York            35                        10.0
...                                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Texas               44                         1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                         5.0
                                                                                                      MVR EDIT REJECT ONLINE                             0214           28362          New York            35                         1.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          Massachusetts       20                         1.0
                                                                                                                                                                                       New York            35                        50.0

[27626 rows x 1 columns]

In [23]:
Prior_WK_4= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-4'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_4.columns= ['Prior_WK_4_Units']
Prior_WK_4

Prior_WK_4_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          Arizona             4                         20.0
                                                                                                                                                                                       California          5                         28.0
                                                                                                                                                                                       Florida             10                         1.0
                                                                                                                                                                                       Hawaii              12                         1.0
                                                                                                                                                                                       Missouri            25                         3.0
...                                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Connecticut         7                          1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          Michigan            23                         1.0
                                                                                                                                                                                       New York            35                         2.0
                                                                                                      MVR EDIT REJECT ONLINE                             0214           28362          Florida             10                         1.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                        54.0

[28961 rows x 1 columns]

In [24]:
Prior_WK_5= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-5'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_5.columns= ['Prior_WK_5_Units']
Prior_WK_5

Prior_WK_5_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          California          5                         34.0
                                                                                                                                                                                       Florida             10                         2.0
                                                                                                                                                                                       Hawaii              12                         2.0
                                                                                                                                                                                       Michigan            23                        19.0
                                                                                                                                                                                       Missouri            25                         3.0
...                                                                                                                                                                                                                                   ...
99005           Thrivent Financial for Lutherans   Thrivent Financial for Lutherans   99005           MVR WEST VIRGINIA 60-MONTH ONLINE                  0488           39711          West Virginia       50                         1.0
                                                                                                      MVR WISCONSIN 60-MONTH CUSTOM ONLINE               2322           29081          Wisconsin           49                        51.0
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Maryland            21                         1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                         2.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                        51.0

[29052 rows x 1 columns]

In [25]:
Prior_WK_6= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-6'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_6.columns= ['Prior_WK_6_Units']
Prior_WK_6

Prior_WK_6_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                     search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT               2048           29007          California          5                         45.0
                                                                                                                                                                         Hawaii              12                         1.0
                                                                                                                                                                         Missouri            25                         4.0
                                                                                                                                                                         New York            35                        16.0
                                                                                                                                                                         North Carolina      28                        38.0
...                                                                                                                                                                                                                     ...
99005           Thrivent Financial for Lutherans   Thrivent Financial for Lutherans   99005           MVR WISCONSIN 60-MONTH CUSTOM ONLINE 2322           29081          Wisconsin           49                        64.0
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR DRIVER HISTORY DATABASE ONLINE   2309           29072          Michigan            23                         1.0
                                                                                                                                                                         New York            35                         1.0
                                                                                                      MVR GEORGIA 36-MONTH NO HIT ONLINE   0303           28405          Georgia             11                         1.0
                                                                                                      MVR ONLINE ACCESS                    0004           28263          New York            35                        41.0

[28945 rows x 1 columns]

In [26]:
Prior_WK_7= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-7'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_7.columns= ['Prior_WK_7_Units']
Prior_WK_7

Prior_WK_7_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          California          5                          1.0
                                                                                                                                                                                       Florida             10                        19.0
                                                                                                                                                                                       Missouri            25                         4.0
                                                                                                                                                                                       North Carolina      28                         3.0
                                                                                                                                                                                       Ohio                36                         9.0
...                                                                                                                                                                                                                                   ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          Texas               44                         1.0
                                                                                                                                                                                       Wisconsin           49                         1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                         4.0
                                                                                                      MVR EDIT REJECT ONLINE                             0214           28362          Georgia             11                         1.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                        43.0

[29152 rows x 1 columns]

In [27]:
Prior_WK_8= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-8'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_8.columns= ['Prior_WK_8_Units']
Prior_WK_8

Prior_WK_8_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                     search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT               2048           29007          Arizona             4                          1.0
                                                                                                                                                                         California          5                         33.0
                                                                                                                                                                         Missouri            25                         4.0
                                                                                                                                                                         New York            35                         2.0
                                                                                                                                                                         North Carolina      28                         4.0
...                                                                                                                                                                                                                     ...
99005           Thrivent Financial for Lutherans   Thrivent Financial for Lutherans   99005           MVR WISCONSIN 60-MONTH CUSTOM ONLINE 2322           29081          Wisconsin           49                        53.0
                                                                                                      MVR WYOMING 60-MONTH ONLINE          0211           28360          Wyoming             51                         1.0
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR DRIVER HISTORY DATABASE ONLINE   2309           29072          New York            35                         5.0
                                                                                                      MVR EDIT REJECT ONLINE               0214           28362          Pennsylvania        39                         1.0
                                                                                                      MVR ONLINE ACCESS                    0004           28263          New York            35                        35.0

[27458 rows x 1 columns]

In [28]:
Prior_WK_9= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-9'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_9.columns= ['Prior_WK_9_Units']
Prior_WK_9

Prior_WK_9_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                     search_type_id search_type_sk state_province_name geography_sk                  
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT               2048           29007          California          5                         16.0
                                                                                                                                                                         Florida             10                         1.0
                                                                                                                                                                         Hawaii              12                        21.0
                                                                                                                                                                         Missouri            25                         1.0
                                                                                                                                                                         New York            35                         4.0
...                                                                                                                                                                                                                     ...
99005           Thrivent Financial for Lutherans   Thrivent Financial for Lutherans   99005           MVR WISCONSIN 60-MONTH CUSTOM ONLINE 2322           29081          Wisconsin           49                        61.0
                                                                                                      MVR WYOMING 60-MONTH ONLINE          0211           28360          Wyoming             51                         1.0
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR DRIVER HISTORY DATABASE ONLINE   2309           29072          New York            35                         4.0
                                                                                                      MVR ONLINE ACCESS                    0004           28263          New York            35                        39.0
                                                                                                      MVR ONLINE ACCESS- NO HIT            0090           28314          New York            35                         1.0

[28444 rows x 1 columns]

In [29]:
Prior_WK_10= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-10'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_10.columns= ['Prior_WK_10_Units']
Prior_WK_10

Prior_WK_10_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name             search_type_id search_type_sk state_province_name geography_sk                   
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT       2048           29007          California          5                           1.0
                                                                                                                                                                 Missouri            25                          4.0
                                                                                                                                                                 North Dakota        29                         23.0
                                                                                                                                                                 Ohio                36                         10.0
                                                                                                      MVR BTI AGENT                2402           29100          Alabama             2                           9.0
...                                                                                                                                                                                                              ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR MISSOURI DATABASE ONLINE 0551           28507          Missouri            25                          1.0
                                                                                                      MVR ONLINE ACCESS            0004           28263          Massachusetts       20                          1.0
                                                                                                                                                                 Michigan            23                          1.0
                                                                                                                                                                 New York            35                         43.0
                                                                                                                                                                 South Carolina      41                          1.0

[28451 rows x 1 columns]

In [30]:
Prior_WK_11= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-11'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_11.columns= ['Prior_WK_11_Units']
Prior_WK_11

Prior_WK_11_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                                   search_type_id search_type_sk state_province_name geography_sk                   
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                             2048           29007          Arizona             4                           2.0
                                                                                                                                                                                       Hawaii              12                          4.0
                                                                                                                                                                                       Missouri            25                          1.0
                                                                                                                                                                                       New York            35                          2.0
                                                                                                                                                                                       North Carolina      28                          3.0
...                                                                                                                                                                                                                                    ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR  DRIVING BEHAVIOR 360 36-MONTH COURT CUSTOM... 6459           58764          New Jersey          32                          1.0
                                                                                                      MVR DRIVER HISTORY DATABASE ONLINE                 2309           29072          New York            35                          5.0
                                                                                                      MVR EDIT REJECT ONLINE                             0214           28362          Pennsylvania        39                          3.0
                                                                                                      MVR INDIANA 36-MONTH CUSTOM ONLINE                 1070           35008          Indiana             16                          1.0
                                                                                                      MVR ONLINE ACCESS                                  0004           28263          New York            35                         49.0

[28735 rows x 1 columns]

In [31]:
Prior_WK_12= pd.DataFrame(data[data['week_format'] == 'Prior-Wk-12'].pivot_table(index=['ins_control_nbr', 'ins_control_name','ins_company_name', 'ins_company_nbr','search_type_name','search_type_id','search_type_sk','state_province_name','geography_sk'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['total_units'],
               aggfunc={'total_units':np.sum}))
Prior_WK_12.columns= ['Prior_WK_12_Units']
Prior_WK_12

Prior_WK_12_Units
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name                          search_type_id search_type_sk state_province_name geography_sk                   
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT                    2048           29007          Arizona             4                          40.0
                                                                                                                                                                              California          5                           4.0
                                                                                                                                                                              Florida             10                          1.0
                                                                                                                                                                              Hawaii              12                          2.0
                                                                                                                                                                              Louisiana           19                          1.0
...                                                                                                                                                                                                                           ...
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR DRIVER HISTORY DATABASE ONLINE        2309           29072          New York            35                          2.0
                                                                                                      MVR EDIT REJECT ONLINE                    0214           28362          Pennsylvania        39                          2.0
                                                                                                      MVR NORTH CAROLINA 36-MONTH CUSTOM ONLINE 5896           34765          North Carolina      28                          1.0
                                                                                                      MVR ONLINE ACCESS                         0004           28263          New York            35                         37.0
                                                                                                      MVR ONLINE ACCESS- NO HIT                 0090           28314          Michigan            23                          1.0

[28547 rows x 1 columns]

In [32]:
### merge 12 weeks calculation 

frames = [current_wk, Prior_WK_1, Prior_WK_2,Prior_WK_3,Prior_WK_4,Prior_WK_5,Prior_WK_6,Prior_WK_7,Prior_WK_8,Prior_WK_9,Prior_WK_10,Prior_WK_11,Prior_WK_12]

In [33]:
result2 = pd.concat(frames,axis=1, sort=False)
result3 = pd.DataFrame(result2).fillna(0)
result3.head()

Current_Wk_units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          0.0   
                                                                                                                       Arkansas            3                          0.0   
                                                                                                                       California          5                          2.0   
                                                                                                                       Florida             10                         2.0   
                                                                                                                       Hawaii              12                         2.0   

                                                                                                                                                         Prior_WK_1_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          0.0   
                                                                                                                       Arkansas            3                          5.0   
                                                                                                                       California          5                          0.0   
                                                                                                                       Florida             10                         0.0   
                                                                                                                       Hawaii              12                         2.0   

                                                                                                                                                         Prior_WK_2_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                         43.0   
                                                                                                                       Arkansas            3                          0.0   
                                                                                                                       California          5                          0.0   
                                                                                                                       Florida             10                         1.0   
                                                                                                                       Hawaii              12                         0.0   

                                                                                                                                                         Prior_WK_3_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          2.0  

In [34]:
result3.shape

(60007, 13)

In [35]:
result4 = result3

In [36]:
result5 = result4.reset_index()

In [37]:
result5.head()

,ins_control_nbr,ins_control_name,ins_company_name,ins_company_nbr,search_type_name,search_type_id,search_type_sk,state_province_name,geography_sk,Current_Wk_units,...,Prior_WK_3_Units,Prior_WK_4_Units,Prior_WK_5_Units,Prior_WK_6_Units,Prior_WK_7_Units,Prior_WK_8_Units,Prior_WK_9_Units,Prior_WK_10_Units,Prior_WK_11_Units,Prior_WK_12_Units
0,00000,Unassigned,Unassigned,00000,MVR BATCH ACCESS AGENT,2048,29007,Arizona,4,0.0,...,2.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,40.0
1,00000,Unassigned,Unassigned,00000,MVR BATCH ACCESS AGENT,2048,29007,Arkansas,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00000,Unassigned,Unassigned,00000,MVR BATCH ACCESS AGENT,2048,29007,California,5,2.0,...,1.0,28.0,34.0,45.0,1.0,33.0,16.0,1.0,0.0,4.0
3,00000,Unassigned,Unassigned,00000,MVR BATCH ACCESS AGENT,2048,29007,Florida,10,2.0,...,0.0,1.0,2.0,0.0,19.0,0.0,1.0,0.0,0.0,1.0
4,00000,Unassigned,Unassigned,00000,MVR BATCH ACCESS AGENT,2048,29007,Hawaii,12,2.0,...,2.0,1.0,2.0,1.0,0.0,0.0,21.0,0.0,4.0,2.0


In [38]:
result4.shape

(60007, 13)

In [ ]:
#results.to_csv('results_MVR_0726.csv', sep='\t', encoding='utf-8')

In [39]:
MVR_final = result3

In [40]:
MVR_final.iloc[:, 1:8]

Prior_WK_1_Units  \
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name          search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT    2048           29007          Arizona             4                          0.0   
                                                                                                                                                              Arkansas            3                          5.0   
                                                                                                                                                              California          5                          0.0   
                                                                                                                                                              Florida             10                         0.0   
                                                                                                                                                              Hawaii              12                         2.0   
...                                                                                                                                                                                                          ...   
99245           Ladder Insurance Services, LLC- HQ Ladder Insurance Services, LLC- HQ 99245           MVR ONLINE ACCESS         0004           28263          New York            35                        62.0   
                                                                                                                                                              South Carolina      41                         0.0   
                                                                                                                                                              Vermont             47                         0.0   
                                                                                                      MVR ONLINE ACCESS- NO HIT 0090           28314          Michigan            23                         0.0   
                                                                                                                                                              New York            35                         0.0   

                                                                                                                                                                                                Prior_WK_2_Units  \
ins_control_nbr ins_control_name                   ins_company_name                   ins_company_nbr search_type_name          search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned                         Unassigned                         00000           MVR BATCH ACCESS AGENT    2048           29007          Arizona             4                         43.0   
                                                                                                                                                              Arkansas            3                          0.0   
                                                                                                                                                              California          5                          0.0   
                                                                                                                                                              Florida             10                         1.0   
                                                                                                                                                              Hawai

In [41]:
MVR_final['tot_7wk_adjunits'] = MVR_final.iloc[:, 1:8].sum(axis=1)
MVR_final.head()

Current_Wk_units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          0.0   
                                                                                                                       Arkansas            3                          0.0   
                                                                                                                       California          5                          2.0   
                                                                                                                       Florida             10                         2.0   
                                                                                                                       Hawaii              12                         2.0   

                                                                                                                                                         Prior_WK_1_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          0.0   
                                                                                                                       Arkansas            3                          5.0   
                                                                                                                       California          5                          0.0   
                                                                                                                       Florida             10                         0.0   
                                                                                                                       Hawaii              12                         2.0   

                                                                                                                                                         Prior_WK_2_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                         43.0   
                                                                                                                       Arkansas            3                          0.0   
                                                                                                                       California          5                          0.0   
                                                                                                                       Florida             10                         1.0   
                                                                                                                       Hawaii              12                         0.0   

                                                                                                                                                         Prior_WK_3_Units  \
ins_control_nbr ins_control_name ins_company_name ins_company_nbr search_type_name       search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned       Unassigned       00000           MVR BATCH ACCESS AGENT 2048           29007          Arizona             4                          2.0  

In [59]:
MVR_final_sm400 = MVR_final[MVR_final['tot_7wk_adjunits'] <400]

In [78]:
MVR_final_sm400.shape

(55360, 14)

In [61]:
MVR_final_sm400.to_csv('MVR_final_sm400.csv', sep='\t', encoding='utf-8')

In [62]:
MVR_final_adj = MVR_final[MVR_final['tot_7wk_adjunits'] >=400] ### only keep dataset with 7 weeks total units >=400

In [63]:
MVR_final_adj                                                                                                                                                                                    

Current_Wk_units  \
ins_control_nbr ins_control_name                               ins_company_name                               ins_company_nbr search_type_name                         search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned                                     Unassigned                                     00000           MVR DRIVER HISTORY DATABASE ONLINE AGENT 0568           28519          New York            35                       101.0   
                                                                                                                                                                                                     Ohio                36                        57.0   
                                                                                                                              MVR MINNESOTA DAILY ONLINE AGENT         1018           28685          Minnesota           24                        32.0   
                                                                                                                              MVR MISSIOURI ONLINE AGENT               0570           28521          Missouri            25                       117.0   
                                                                                                                              MVR NO HIT ONLINE AGENT                  2066           29017          California          5                         30.0   
...                                                                                                                                                                                                                                                 ...   
98455           Allianz Global Risks US Insurance Company      Fireman'S Fund Cos                             03032           MVR ONLINE ACCESS                        0004           28263          Texas               44                        76.0   
                                                                                                                                                                                                     Washington          48                        83.0   
98715           Farm Bureau Life Insurance Company of Michigan Farm Bureau Life Insurance Company of Michigan 98715           MVR ONLINE ACCESS                        0004           28263          Michigan            23                       123.0   
99005           Thrivent Financial for Lutherans               Thrivent Financial for Lutherans               99005           MVR MINNESOTA DATABASE DAILY ONLINE      0494           28478          Minnesota           24                        78.0   
                                                                                                                              MVR WISCONSIN 60-MONTH CUSTOM ONLINE     2322           29081          Wisconsin           49                        66.0   

                                                                                                                                                                                                                                       Prior_WK_1_Units  \
ins_control_nbr ins_control_name                               ins_company_name                               ins_company_nbr search_type_name                         search_type_id search_type_sk state_province_name geography_sk                     
00000           Unassigned                                     Unassigned                                     00000           MVR DRIVER HISTORY DATABASE ONLINE AGENT 0568           28519          New York            35                        72.0   
                                                                                                                                                                                                     Ohio             

In [64]:
MVR_final_adj['12_wk_std'] = MVR_final_adj.iloc[:, 1:13].std(axis=1)

<ipython-input-64-d66e847abf95>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MVR_final_adj['12_wk_std'] = MVR_final_adj.iloc[:, 1:13].std(axis=1)


In [65]:
MVR_final_adj['12_wk_mean'] = MVR_final_adj.iloc[:, 1:13].mean(axis=1)

<ipython-input-65-38c7e57e7e9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MVR_final_adj['12_wk_mean'] = MVR_final_adj.iloc[:, 1:13].mean(axis=1)


In [68]:
MVR_final_adj['12_wk_std_4x'] = MVR_final_adj['12_wk_mean']+MVR_final_adj['12_wk_std']*4

<ipython-input-68-b3215ad234f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MVR_final_adj['12_wk_std_4x'] = MVR_final_adj['12_wk_mean']+MVR_final_adj['12_wk_std']*4


In [69]:
len(MVR_final_adj['Current_Wk_units'][MVR_final_adj['Current_Wk_units'] >MVR_final_adj['12_wk_std_4x']])

26

In [71]:
results_alert = MVR_final_adj[MVR_final_adj['Current_Wk_units'] >MVR_final_adj['12_wk_std_4x']]


In [73]:
results_alert.shape

(26, 17)

In [74]:
results_non_alert = MVR_final_adj[MVR_final_adj['Current_Wk_units'] <=MVR_final_adj['12_wk_std_4x']]

In [75]:
results_non_alert.shape

(4621, 17)

In [76]:
results_alert.to_csv('results_alert.csv', sep='\t', encoding='utf-8')

In [77]:
results_non_alert.to_csv('results_non_alert.csv', sep='\t', encoding='utf-8')

In [52]:
cols = [8,9,10,11,12,14,15,16]
results.drop(results.columns[cols],axis=1,inplace=True)

/var/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [53]:
results.head()

Current_Wk_units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       401.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       312.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                       154.0   
                                                                                                                                           Texas               44                      1232.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      8995.0   

                                                                                                                                                                             Prior_WK_1_Units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       187.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       184.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                       101.0   
                                                                                                                                           Texas               44                      1019.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      7557.0   

                                                                                                                                                                             Prior_WK_2_Units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       255.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       229.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                        86.0   
                                                                                                                                           Texas               44                       980.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      7260.0   

                                                                                                                     

In [54]:
results['%Diff(2-WK-Avg)'] = (results['Current_Wk_units'] - results.iloc[:, 1:3].mean(axis = 1))/results.iloc[:, 1:3].mean(axis = 1) *100

<ipython-input-54-d8e0f3385ae4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['%Diff(2-WK-Avg)'] = (results['Current_Wk_units'] - results.iloc[:, 1:3].mean(axis = 1))/results.iloc[:, 1:3].mean(axis = 1) *100


In [55]:
results.iloc[:, 1:3]

Prior_WK_1_Units  \
ins_control_nbr ins_control_name                            ins_company_name                            ins_company_nbr search_type_name                          search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp                              Sentry Ins Grp                              03036           MVR ONLINE ACCESS                         0004           28263          Texas               44                       187.0   
03083           State Farm Mut Auto Ins                     State Farm Mut                              03071           MVR DRIVER HISTORY BATCH                  0545           28502          Texas               44                       184.0   
                                                                                                                        MVR DRIVER HISTORY DATABASE ONLINE        2309           29072          Illinois            15                       101.0   
                                                                                                                                                                                                Texas               44                      1019.0   
                                                                                                                        MVR ILLINOIS 36-MONTH CUSTOM ONLINE       5267           29709          Illinois            15                      7557.0   
03269           Ngm Ins Co                                  Ngm Ins Co                                  03269           MVR FLORIDA 36-MONTH CUSTOM               5494           29719          Florida             10                       562.0   
03341           Discovery Ins                               Discovery Ins                               03341           MVR NORTH CAROLINA 12-MONTH CUSTOM ONLINE 5895           36996          North Carolina      28                       699.0   
03362           Amica Mut Grp                               Amica Mut Grp                               03362           MVR NORTH CAROLINA 36-MONTH CUSTOM ONLINE 5896           34765          North Carolina      28                       386.0   
03468           Cincinnati Ins Cos                          Cincinnati Ins Cos                          03468           MVR INDIANA 12-MONTH CUSTOM               1064           35088          Indiana             16                       170.0   
03565           American Family Ins Grp                     Permanent Gen Cos                           03295           MVR BATCH ACCESS                          0003           28262          North Carolina      28                       249.0   
03566           Ameriprise Finl                             AmFam Connect Property and Casualty         03878           MVR ILLINOIS 36-MONTH CUSTOM ONLINE       5267           29709          Illinois            15                        74.0   
03569           Amern Nat P&C Grp                           Amern Nat P&C                               03421           MVR SOUTH CAROLINA 6-MONTH                0391           28440          South Carolina      41                       863.0   
03583           Metlife Inc                                 Metropolitan P&C Ins Cos                    03420           MVR BATCH ACCESS                          0003           28262          California          5                        152.0   
03585           Nationwide Grp                              Nationwide Cos                              03152           MVR ONLINE ACCESS                         0004           28263          Alabama             2                         88.0   
                                                                                                                                                                                                Illinois            15                        77.0   
                                            

In [56]:
results.head()

Current_Wk_units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       401.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       312.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                       154.0   
                                                                                                                                           Texas               44                      1232.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      8995.0   

                                                                                                                                                                             Prior_WK_1_Units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       187.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       184.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                       101.0   
                                                                                                                                           Texas               44                      1019.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      7557.0   

                                                                                                                                                                             Prior_WK_2_Units  \
ins_control_nbr ins_control_name        ins_company_name ins_company_nbr search_type_name                    search_type_id search_type_sk state_province_name geography_sk                     
03036           Sentry Ins Grp          Sentry Ins Grp   03036           MVR ONLINE ACCESS                   0004           28263          Texas               44                       255.0   
03083           State Farm Mut Auto Ins State Farm Mut   03071           MVR DRIVER HISTORY BATCH            0545           28502          Texas               44                       229.0   
                                                                         MVR DRIVER HISTORY DATABASE ONLINE  2309           29072          Illinois            15                        86.0   
                                                                                                                                           Texas               44                       980.0   
                                                                         MVR ILLINOIS 36-MONTH CUSTOM ONLINE 5267           29709          Illinois            15                      7260.0   

                                                                                                                     

In [57]:
results.reset_index()

,ins_control_nbr,ins_control_name,ins_company_name,ins_company_nbr,search_type_name,search_type_id,search_type_sk,state_province_name,geography_sk,Current_Wk_units,Prior_WK_1_Units,Prior_WK_2_Units,Prior_WK_3_Units,Prior_WK_4_Units,Prior_WK_5_Units,Prior_WK_6_Units,Prior_WK_7_Units,tot_7wk_adjunits,%Diff(2-WK-Avg)
0,03036,Sentry Ins Grp,Sentry Ins Grp,03036,MVR ONLINE ACCESS,0004,28263,Texas,44,401.0,187.0,255.0,157.0,163.0,156.0,170.0,202.0,1290.0,81.447964
1,03083,State Farm Mut Auto Ins,State Farm Mut,03071,MVR DRIVER HISTORY BATCH,0545,28502,Texas,44,312.0,184.0,229.0,240.0,220.0,209.0,227.0,240.0,1549.0,51.089588
2,03083,State Farm Mut Auto Ins,State Farm Mut,03071,MVR DRIVER HISTORY DATABASE ONLINE,2309,29072,Illinois,15,154.0,101.0,86.0,80.0,85.0,91.0,93.0,96.0,632.0,64.705882
3,03083,State Farm Mut Auto Ins,State Farm Mut,03071,MVR DRIVER HISTORY DATABASE ONLINE,2309,29072,Texas,44,1232.0,1019.0,980.0,892.0,976.0,970.0,956.0,999.0,6792.0,23.261631
4,03083,State Farm Mut Auto Ins,State Farm Mut,03071,MVR ILLINOIS 36-MONTH CUSTOM ONLINE,5267,29709,Illinois,15,8995.0,7557.0,7260.0,6232.0,6859.0,7108.0,7401.0,6949.0,49366.0,21.414591
5,03269,Ngm Ins Co,Ngm Ins Co,03269,MVR FLORIDA 36-MONTH CUSTOM,5494,29719,Florida,10,3123.0,562.0,443.0,786.0,1752.0,260.0,572.0,646.0,5021.0,521.492537
6,03341,Discovery Ins,Discovery Ins,03341,MVR NORTH CAROLINA 12-MONTH CUSTOM ONLINE,5895,36996,North Carolina,28,955.0,699.0,704.0,640.0,678.0,533.0,572.0,577.0,4403.0,36.136850
7,03362,Amica Mut Grp,Amica Mut Grp,03362,MVR NORTH CAROLINA 36-MONTH CUSTOM ONLINE,5896,34765,North Carolina,28,497.0,386.0,385.0,348.0,348.0,293.0,336.0,292.0,2388.0,28.923476
8,03468,Cincinnati Ins Cos,Cincinnati Ins Cos,03468,MVR INDIANA 12-MONTH CUSTOM,1064,35088,Indiana,16,1482.0,170.0,144.0,176.0,615.0,197.0,235.0,224.0,1761.0,843.949045
9,03565,American Family Ins Grp,Permanent Gen Cos,03295,MVR BATCH ACCESS,0003,28262,North Carolina,28,452.0,249.0,290.0,223.0,306.0,216.0,208.0,229.0,1721.0,67.717996


In [58]:
results.to_csv('results_MVR_0726.csv', sep='\t', encoding='utf-8')

In [80]:
MVR_final_sm400 = MVR_final_sm400.reset_index()

In [81]:
for col in MVR_final_sm400.columns:
    print(col)

ins_control_nbr
ins_control_name
ins_company_name
ins_company_nbr
search_type_name
search_type_id
search_type_sk
state_province_name
geography_sk
Current_Wk_units
Prior_WK_1_Units
Prior_WK_2_Units
Prior_WK_3_Units
Prior_WK_4_Units
Prior_WK_5_Units
Prior_WK_6_Units
Prior_WK_7_Units
Prior_WK_8_Units
Prior_WK_9_Units
Prior_WK_10_Units
Prior_WK_11_Units
Prior_WK_12_Units
tot_7wk_adjunits


In [84]:
MVR_final_sm400.columns = ['ins_control_nbr', 'ins_control_name', 'ins_company_name',
       'ins_company_nbr', 'search_type_name', 'search_type_id',
       'search_type_sk', 'state_province_name', 'geography_sk',
       'field_1_curr_freq_value', 'field_1_prior_freq_1_value', 'field_1_prior_freq_2_value',
       'field_1_prior_freq_3_value', 'field_1_prior_freq_4_value', 'field_1_prior_freq_5_value',
       'field_1_prior_freq_6_value', 'field_1_prior_freq_7_value', 'field_1_prior_freq_8_value',
       'field_1_prior_freq_9_value', 'field_1_prior_freq_10_value', 'field_1_prior_freq_11_value',
       'field_1_prior_freq_12_value', 'tot_7wk_adjunits']

In [114]:
df1 = pd.read_csv("vijay_sm_400.csv", encoding = 'unicode_escape')

In [115]:
df1.head()

,ï»¿wuid,date_sk,geography_sk,alert_frequency_sk,product_sk,search_type_sk,claim_type_disp_sk,activity_file_rollup,alert_run_date_sk,alert_type_sk,...,field_3_prior_freq_3_value,field_3_prior_freq_4_value,field_3_prior_freq_5_value,field_3_prior_freq_6_value,field_3_prior_freq_7_value,action_date_sk,due_date_sk,row_id,alert_flag,filter_out_reason
0,W20201014-093824,20200726,1,2,84,29017,-1,NaN,20201014,4,...,0.0,0.0,0.0,0.0,0.0,0,0,1,NaN,total_7_weeks units < 400
1,W20201014-093824,20200726,1,2,84,29008,-1,NaN,20201014,4,...,0.0,0.0,0.0,0.0,0.0,0,0,60009,NaN,total_7_weeks units < 400
2,W20201014-093824,20200726,1,2,84,28322,-1,NaN,20201014,4,...,0.0,0.0,0.0,0.0,0.0,0,0,3,NaN,total_7_weeks units < 400
3,W20201014-093824,20200726,1,2,84,28362,-1,NaN,20201014,4,...,0.0,0.0,0.0,0.0,0.0,0,0,4,NaN,total_7_weeks units < 400
4,W20201014-093824,20200726,1,2,84,40049,-1,NaN,20201014,4,...,0.0,0.0,0.0,0.0,0.0,0,0,5,NaN,total_7_weeks units < 400


In [122]:
df2 = MVR_final_sm400

In [137]:
type(df2)

pandas.core.frame.DataFrame

In [138]:
type(df1)

pandas.core.frame.DataFrame

In [125]:
df3 = df2.replace(np.nan, 0)

In [162]:
df3.to_csv('MVR_final_sm400.csv', sep='\t', encoding='utf-8')

In [146]:
df4 = pd.read_csv("python_final_sm400.csv", encoding = 'unicode_escape')

In [147]:
compare = datacompy.Compare( 
    df1, 
    df4, 
      
    # You can also specify a list 
    # of columns 
    join_columns = ['geography_sk','search_type_sk','ins_control_nbr', 'ins_control_name', 'ins_company_nbr', 'ins_company_name']

      
    # Optional, defaults to 0 
    #abs_tol = 0, 
      
    # Optional, defaults to 0 
    #rel_tol = 0,  
      
    # Optional, defaults to 'df1' 
    #df1_name = 'Original', 
      
    # Optional, defaults to 'df2' 
   # df2_name = 'New' 
    ) 
  

In [148]:
print(compare.report()) 

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0       df1       62  55360
1       df2       22  55360

Column Summary
--------------

Number of columns in common: 22
Number of columns in df1 but not in df2: 40
Number of columns in df2 but not in df1: 0

Row Summary
-----------

Matched on: geography_sk, search_type_sk, ins_control_nbr, ins_control_name, ins_company_nbr, ins_company_name
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 55,360
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 55,360

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 22
Total number of values which compare unequal: 0

Columns with Unequal Values or Types
---------------

In [149]:
results_non_alert.to_csv('MVR_final_non-alert.csv', sep='\t', encoding='utf-8')

In [163]:
df5 = pd.read_csv("vijay_non_alert.csv", encoding = 'unicode_escape')

In [164]:
df6 = pd.read_csv("python_final_non_alert.csv", encoding = 'unicode_escape')

In [165]:
compare = datacompy.Compare( 
    df5, 
    df6, 
      
    # You can also specify a list 
    # of columns 
    join_columns = ['geography_sk','search_type_sk','ins_control_nbr', 'ins_control_name', 'ins_company_nbr', 'ins_company_name']

      
    # Optional, defaults to 0 
    #abs_tol = 0, 
      
    # Optional, defaults to 0 
    #rel_tol = 0,  
      
    # Optional, defaults to 'df1' 
    #df1_name = 'Original', 
      
    # Optional, defaults to 'df2' 
   # df2_name = 'New' 
    ) 

In [166]:
print(compare.report()) 

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1       62  4621
1       df2       26  4621

Column Summary
--------------

Number of columns in common: 6
Number of columns in df1 but not in df2: 56
Number of columns in df2 but not in df1: 20

Row Summary
-----------

Matched on: geography_sk, search_type_sk, ins_control_nbr, ins_control_name, ins_company_nbr, ins_company_name
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 4,598
Number of rows in df1 but not in df2: 23
Number of rows in df2 but not in df1: 23

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 4,598

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 6
Total number of values which compare unequal: 0

Sample Rows Only in df1 (First 10 Columns)
-------------

In [167]:
compare.report()

'DataComPy Comparison\n--------------------\n\nDataFrame Summary\n-----------------\n\n  DataFrame  Columns  Rows\n0       df1       62  4621\n1       df2       26  4621\n\nColumn Summary\n--------------\n\nNumber of columns in common: 6\nNumber of columns in df1 but not in df2: 56\nNumber of columns in df2 but not in df1: 20\n\nRow Summary\n-----------\n\nMatched on: geography_sk, search_type_sk, ins_control_nbr, ins_control_name, ins_company_nbr, ins_company_name\nAny duplicates on match values: No\nAbsolute Tolerance: 0\nRelative Tolerance: 0\nNumber of rows in common: 4,598\nNumber of rows in df1 but not in df2: 23\nNumber of rows in df2 but not in df1: 23\n\nNumber of rows with some compared columns unequal: 0\nNumber of rows with all compared columns equal: 4,598\n\nColumn Comparison\n-----------------\n\nNumber of columns compared with some values unequal: 0\nNumber of columns compared with all values equal: 6\nTotal number of values which compare unequal: 0\n\nSample Rows Only 